### Imports

In [10]:
import pandas as pd

# import GCP Demo 1 package
from gcpdemo1 import gcp, etl, data, tune, train, predict

### ***USER INPUT***: Enter GCP Variables

In [2]:
# GCP variables
gcp_vars = {
    'credentials_path': 'C:/Users/kendall.ocasio/gcp-demo1/credentials/ml-sandbox-1-191918-4714b5fd6e92.json', # local path to gcp credentials
    'project_name': 'ml-sandbox-1-191918', # gcp project name
    'bucket': 'gcp-cert-demo-1', # gcp bucket name
    'gcs_trainer_path': 'taxi_mlp_trainer/trainer-0.1.tar.gz', # path to write trainer package to within bucket
    'local_trainer_path': '../mlp_trainer', # local path to trainer package
    'table_id': 'finaltaxi_encoded_sampled_small' # bigquery table to use as dataset TODO: temporary small table
}

### Authentication

In [3]:
# get GCP credentials
gcp_credentials = gcp.get_credentials(gcp_vars['credentials_path'])

### Create training package

In [ ]:
# build and upload trainer package
# TODO: does not work on windows because of no bash command
gcpdemo1.gcp.build_and_upload_trainer_package(
    bucket_name=gcp_vars['bucket'],
    destination_blob_name=gcp_vars['gcs_trainer_path'],
    local_trainer_package_path=gcp_vars['local_trainer_path'],
    credentials=gcp_credentials
)

### Tune hyperparameters

In [ ]:
# TODO: tune here

### ***USER INPUT***: Enter training and job parameters

In [4]:
# input training parameters
training_params = {
    'dense_neurons_1': 64,
    'dense_neurons_2': 32,
    'dense_neurons_3': 8,
    'activation': 'relu',
    'dropout_rate_1': 0.1,
    'dropout_rate_2': 0.1,
    'dropout_rate_3': 0.1,
    'optimizer': 'adam',
    'learning_rate': 0.1,
    'chunk_size': 500000,
    'batch_size': 1024,
    'epochs': 3,
    'validation_freq': 5,
    'kernel_initial_1': 'normal',
    'kernel_initial_2': 'normal',
    'kernel_initial_3': 'normal',
}

# input job parameters
job_params = {
    'job_id': 'taxi_mlp_trainer', # training job ID
    'job_dir': 'taxi_mlp_model', # model output directory
    'model_name': 'taxi_mlp_model' # model name for deployment
}

### Train model

In [ ]:
# create mlp trainer
mlp_trainer = train.MLPTrainer(
    credentials=gcp_credentials,
    project_name=gcp_vars['project_name'],
    bucket=gcp_vars['bucket'],
    table_id=gcp_vars['table_id']
)

# begin training job
mlp_trainer.train(
    job_id=job_params['job_id'], 
    job_dir=job_params['job_dir'],
    package_uri=gcp_vars['gcs_trainer_path'],
    dense_neurons_1=training_params['dense_neurons_1'], 
    dense_neurons_2=training_params['dense_neurons_2'], 
    dense_neurons_3=training_params['dense_neurons_3'], 
    activation=training_params['activation'],
    dropout_rate_1=training_params['dropout_rate_1'], 
    dropout_rate_2=training_params['dropout_rate_2'], 
    dropout_rate_3=training_params['dropout_rate_3'], 
    optimizer=training_params['optimizer'], 
    learning_rate=training_params['learning_rate'], 
    chunk_size=training_params['chunk_size'], 
    batch_size=training_params['batch_size'], 
    epochs=training_params['epochs'],
    validation_freq=training_params['validation_freq'], 
    kernel_initial_1=training_params['kernel_initial_1'], 
    kernel_initial_2=training_params['kernel_initial_2'], 
    kernel_initial_3=training_params['kernel_initial_3'],
)

### Check training status

In [ ]:
# check status of training job
mlp_trainer.training_status()

### Deploy model

In [ ]:
# deploy trained model
mlp_trainer.deploy(job_params['model_name'])

### Make predictions

In [13]:
# TODO: basic predictions, need to flesh out
# get test data
rows = data.get_reader_rows(
    credentials=gcp_credentials,
    table_id=gcp_vars['table_id'],
    partition_name='test'
)
vals = pd.DataFrame(list(rows)[:10]).drop(
    'cash',
    axis=1
).values
instances = []
for instance in vals:
    instances.append({'dense_input': list(val)})

# predict using deployed model
predictor = predict.Predictor(
    credentials=gcp_credentials,
    project=gcp_vars['project_name'],
    model=mlp_trainer.model_dir
)
predictions = predictor.predict(instances)

2019-11-07 23:34:49,762 [INFO]  URL being requested: GET https://www.googleapis.com/discovery/v1/apis/ml/v1/rest
2019-11-07 23:34:50,139 [INFO]  Predicting using model "mlp_model_1573157868" version "None"
2019-11-07 23:34:50,143 [INFO]  URL being requested: POST https://ml.googleapis.com/v1/projects/ml-sandbox-1-191918/models/mlp_model_1573157868:predict?alt=json
2019-11-07 23:34:50,341 [INFO]  Prediction response:


{'predictions': [{'dense_3': [0.6001867055892944]},
                 {'dense_3': [0.6001867055892944]},
                 {'dense_3': [0.6001867055892944]},
                 {'dense_3': [0.6001867055892944]},
                 {'dense_3': [0.6001867055892944]},
                 {'dense_3': [0.6001867055892944]},
                 {'dense_3': [0.6001867055892944]},
                 {'dense_3': [0.6001867055892944]},
                 {'dense_3': [0.6001867055892944]},
                 {'dense_3': [0.6001867055892944]}]}
